# Week 5 - SQL Corner

* Tables: https://www.sqlitetutorial.net/sqlite-create-table/
* Views: https://www.sqlitetutorial.net/sqlite-create-view/

SQLite Views

* https://www.quora.com/What-is-the-most-dangerous-command-in-SQL
* https://www.quora.com/Whats-the-craziest-SQL-query-youve-ever-encountered



In [1]:
import sqlite3

connection = sqlite3.connect('looneylms.db') # sqlite will create looneylms.db if it doesn't exist, 
                                             # a file now in the same folder as this file

cursor = connection.cursor()
cursor.execute("PRAGMA foreign_keys=ON");
print(connection.total_changes)

0


In [2]:
# multiline strings in Python can be sorrounded with ''' Multi line text '''
query = '''
CREATE TABLE IF NOT EXISTS people (
    id INTEGER PRIMARY KEY, 
    firstname TEXT, 
    lastname TEXT, 
    year INTEGER
)
'''
cursor.execute(query)

In [3]:
cursor.execute("INSERT INTO people(firstname, lastname, year) VALUES ('Bugs', 'Bunny', 1)")
cursor.execute("INSERT INTO people(firstname, lastname, year) VALUES ('Fifi', 'La Fume', 3)")
cursor.execute("INSERT INTO people(firstname, lastname, year) VALUES ('Daffy', 'Duck', 2)")
cursor.execute("INSERT INTO people(firstname, lastname, year) VALUES ('Foghorn', 'Leghorn', 1)")
cursor.execute("INSERT INTO people(firstname, lastname, year) VALUES ('Tweety', 'Bird', 4)")
cursor.execute("INSERT INTO people(firstname, lastname, year) VALUES ('Porky', 'Pig', 4)")

In [4]:
rows = cursor.execute("SELECT * FROM people").fetchall()
print(rows)

[(1, 'Bugs', 'Bunny', 1), (2, 'Fifi', 'La Fume', 3), (3, 'Daffy', 'Duck', 2), (4, 'Foghorn', 'Leghorn', 1), (5, 'Tweety', 'Bird', 4), (6, 'Porky', 'Pig', 4)]


In [5]:
query = '''
CREATE TABLE IF NOT EXISTS quizzes (
    id INTEGER PRIMARY KEY, 
    quizname TEXT, 
    description TEXT
)
'''
cursor.execute(query)

In [6]:
cursor.execute("INSERT INTO quizzes(quizname, description) VALUES ('Looney', 'quiz about laughing')")
cursor.execute("INSERT INTO quizzes(quizname, description) VALUES ('Tune', 'quiz about tunes')")

In [7]:
rows = cursor.execute("SELECT * FROM quizzes").fetchall()
print(rows)

[(1, 'Looney', 'quiz about laughing'), (2, 'Tune', 'quiz about tunes')]


In [8]:
query = '''
CREATE TABLE IF NOT EXISTS quiz_details (
    id INTEGER PRIMARY KEY, 
    quizzes_id INTEGER, 
    items INTEGER,
    FOREIGN KEY (quizzes_id) 
      REFERENCES quizzes (id) 
         ON DELETE CASCADE 
         ON UPDATE CASCADE
)
'''
cursor.execute(query)

In [9]:
cursor.execute("INSERT INTO quiz_details(quizzes_id, items) VALUES (1, 10)")
cursor.execute("INSERT INTO quiz_details(quizzes_id, items) VALUES (2, 10)")

In [10]:
rows = cursor.execute("SELECT * FROM quiz_details").fetchall()
print(rows)

[(1, 1, 10), (2, 2, 10)]


In [11]:
query = '''
CREATE TABLE IF NOT EXISTS quiz_results (
    id INTEGER PRIMARY KEY, 
    quizzes_id INTEGER, 
    people_id INTEGER,
    score INTEGER,
    FOREIGN KEY (people_id) 
      REFERENCES people (id) 
         ON DELETE CASCADE 
         ON UPDATE CASCADE,
    FOREIGN KEY (quizzes_id) 
      REFERENCES quizzes (id) 
         ON DELETE CASCADE 
         ON UPDATE CASCADE
)
'''
cursor.execute(query)

In [12]:
cursor.execute("INSERT INTO quiz_results(quizzes_id, people_id, score) VALUES (1, 3, 7)")
cursor.execute("INSERT INTO quiz_results(quizzes_id, people_id, score) VALUES (2, 5, 8)")
cursor.execute("INSERT INTO quiz_results(quizzes_id, people_id, score) VALUES (2, 1, 9)")
cursor.execute("INSERT INTO quiz_results(quizzes_id, people_id, score) VALUES (1, 4, 8)")
cursor.execute("INSERT INTO quiz_results(quizzes_id, people_id, score) VALUES (2, 3, 7)")
cursor.execute("INSERT INTO quiz_results(quizzes_id, people_id, score) VALUES (1, 2, 8)")

In [13]:
rows = cursor.execute("SELECT * FROM quiz_results").fetchall()
print(rows)

[(1, 1, 3, 7), (2, 2, 5, 8), (3, 2, 1, 9), (4, 1, 4, 8), (5, 2, 3, 7), (6, 1, 2, 8)]


## Views

In [14]:
query = '''
SELECT 
    quizzes.quizname as quiz, 
    people.firstname as firstname, 
    people.lastname as lastname, 
    quiz_results.score as score, 
    people.year as year
FROM 
    quiz_results
INNER JOIN people ON people.id = quiz_results.people_id
INNER JOIN quizzes ON quizzes.id = quiz_results.quizzes_id
ORDER BY quizzes.id;
'''
results = cursor.execute(query).fetchall()
for r in results:
    print(r)

('Looney', 'Daffy', 'Duck', 7, 2)
('Looney', 'Foghorn', 'Leghorn', 8, 1)
('Looney', 'Fifi', 'La Fume', 8, 3)
('Tune', 'Tweety', 'Bird', 8, 4)
('Tune', 'Bugs', 'Bunny', 9, 1)
('Tune', 'Daffy', 'Duck', 7, 2)


In [15]:
query = '''
CREATE VIEW quiz_takers
AS
SELECT 
    quizzes.quizname as quiz, 
    people.firstname as firstname, 
    people.lastname as lastname, 
    quiz_results.score as score, 
    people.year as year
FROM 
    quiz_results
INNER JOIN people ON people.id = quiz_results.people_id
INNER JOIN quizzes ON quizzes.id = quiz_results.quizzes_id
ORDER BY quizzes.id;
'''
cursor.execute(query)

In [16]:
rows = cursor.execute("SELECT * FROM quiz_takers").fetchall()
for r in rows:
    print(r)

('Looney', 'Daffy', 'Duck', 7, 2)
('Looney', 'Foghorn', 'Leghorn', 8, 1)
('Looney', 'Fifi', 'La Fume', 8, 3)
('Tune', 'Tweety', 'Bird', 8, 4)
('Tune', 'Bugs', 'Bunny', 9, 1)
('Tune', 'Daffy', 'Duck', 7, 2)


## Using Pandas

In [17]:
# https://datacarpentry.org/python-ecology-lesson/09-working-with-sql/index.html
import pandas as pd

df = pd.read_sql_query("SELECT * FROM quiz_takers", connection)
print(df.head())

     quiz firstname lastname  score  year
0  Looney     Daffy     Duck      7     2
1  Looney   Foghorn  Leghorn      8     1
2  Looney      Fifi  La Fume      8     3
3    Tune    Tweety     Bird      8     4
4    Tune      Bugs    Bunny      9     1


In [18]:
from contextlib import closing

with closing(sqlite3.connect("looneylms.db")) as connection:
    with closing(connection.cursor()) as cursor:
        rows = cursor.execute("SELECT 1").fetchall()
        print(rows)

[(1,)]
